Data Wrangling
=======

### Initial wrangling

The data for this project was selected as a SQL-query from the EFO product database. Because of the size of the database and the extent of the query, it was decided that this will not be a part of this project. The first draft of the query (before optimization) was estimated to run for about 30 hours in order to give all the ouput. There were then several rounds of optimization, and the final query ran for about eight minutes on a regular computer.

There are different states that a product goes through in the database. It can be a draft, active, retired or a handfull of other states. The output of the initial query included all active products from the database and all features. This is a dataframe with 238 049 active products, each with 125 features.

In [1]:
# Import packages
import matplotlib.pyplot as plt
import os
import pandas as pd
import pickle

# Change working directory to get the file
os.chdir('/Users/pkg/Springboard')

# Read file and set working directory
df = pd.read_csv('Resultat.csv', sep=';')
os.chdir('/Users/pkg/Springboard/Intermediate Data Science with Python/Python_Capstone')

# Shape of the dataframe
df.shape

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2,3,4,7,10,11,12,13,18,19,20,22,24,25,26,30,31,32,33,34,35,36,38,40,41,43,44,47,48,49,50,51,52,54,56,58,60,61,62,63,64,65,66,67,68,69,72,73,74,77,80,81,82,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(238049, 125)

The feature ProductNumber is the unique ID (Elnummer) that all products have, and that will be used to identify all products. The ProductNumber also have an interesting property; the first two digits describe with product category the product belongs in. For instance, if the first two digits are "10", the product is either a cable or a product related to cables. This is stored as a separate feature called "ENG" (Elnummergruppe).

In [2]:
# Set the product number as index
df = df.set_index('ProductNumber')

# Only keep the necessary columns for the project and change the column names
df = df[['ETIM klasse','Teknisk beskrivelse']]
df.columns = ['ETIM_class','Technical_description']
df['ENG'] = df.index.astype(str).str[0:2]
df = df[['ENG','ETIM_class','Technical_description']]

After isolating the pertinent features for the prediction, the dataframe is considearbly smaller.

In [3]:
# Shape of the smaller dataframe
print(df.shape)
df.head()

(238049, 3)


,ENG,ETIM_class,Technical_description
ProductNumber,,,
1000000,10,EC003251,VDF/EMC frekvensomformerkabel med symmetrisk j...
1000001,10,EC003251,VDF/EMC frekvensomformerkabel med symmetrisk j...
1000003,10,EC003251,VDF/EMC frekvensomformerkabel med symmetrisk j...
1000004,10,EC003251,VDF/EMC frekvensomformerkabel med symmetrisk j...
1000005,10,EC003251,VDF/EMC frekvensomformerkabel med symmetrisk j...


Let's see how many missing values there are.

In [4]:
# Number of missing values in the index and columns
print(
    df.index.isnull().sum(),
    df.ENG.isnull().sum(),
    df.ETIM_class.isnull().sum(),
    df.Technical_description.isnull().sum()
)

# Examples of rows with missing values
print(df[df.ETIM_class.isnull()].head())
df[df.Technical_description.isnull()]

0 0 35830 3
              ENG ETIM_class  \
ProductNumber                  
1000015        10        NaN   
1001035        10        NaN   
1001036        10        NaN   
1001051        10        NaN   
1001052        10        NaN   

                                           Technical_description  
ProductNumber                                                     
1000015        VDF/EMC frekvensomformerkabel med symmetrisk j...  
1001035              Elektrisk håndkletørker for kjøkken og bad.  
1001036                         Elektrisk håndkletørker for bad.  
1001051        TFP varmekabelmatte   Lavtbyggende varmekabelm...  
1001052        TFP varmekabelmatte   Lavtbyggende varmekabelm...  


,ENG,ETIM_class,Technical_description
ProductNumber,,,
2462085,24,EC002027,NaN
2462086,24,EC002027,NaN
2462091,24,EC002620,NaN


The ProductNumber has no missing values - good. There are automatically no missing values in the ENG feature, as this was created from the ProductNumber feature. The ETIM class has some missing values, which won't help to predict other ETIM classes. These will simply be removed here, but they could be an interesting "test" group later. There are three products that don't have a technical description. These has already been fixed in the product database, but as they're neglible for this project, they will be removed.

In [5]:
df = df.dropna()
df.shape

(202216, 3)

This leaves in excess of 200 000 products with no missing values, which should be sufficient.

In [6]:
# Set working directory and save the data set as a pickled object.
os.chdir('/Users/pkg/Springboard')
with open('data_wrangling.pickle', 'wb') as handle:
    pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)